In [4]:
import geopandas as gpd
import math
import matplotlib.pyplot as plt
import numpy as np
import warnings

from IPython.display import Javascript, display, clear_output
from ipywidgets import widgets
from scipy import stats
from shapely.geometry import shape, Polygon
from ipyleaflet import (
     Map,
     basemaps,
     DrawControl,
     GeoData,
)

In [36]:
def filter_coords(dc, df):
    #ignore future version err
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        #map based filter
        b = dc['coordinates'][0]
        poly = Polygon(b[:-1])
        clipped = df.clip(poly)
        coords = clipped.get_coordinates().to_numpy().tolist()
    return coords 

#fill up line features with points (increase complexity)
def fill_between_points(pointa, pointb,n):
    (xmin,ymin) = pointa
    (xmax,ymax) = pointb
    return [[xmin + i*(xmax-xmin)/n,ymin + i*(ymax-ymin)/n] for i in range(n)] + [[xmax,ymax]]

def fill_set(coords):
    new = []
    for i in range(len(coords)-1):
        new.extend(fill_between_points(coords[i], coords[i+1], 500))
    print("Set size:",len(new))
    return new

def generate_bin_edges(bounds,l):
    
    (xrange,yrange) = bounds
    (xmin,xmax) = xrange
    (ymin,ymax) = yrange
    
    nx = (xmax-xmin)/l
    ny = (ymax-ymin)/l
    
    xlocs = np.linspace(xmin,xmax-l,math.ceil(nx))
    ylocs = np.linspace(ymin,ymax-l,math.ceil(ny))
            
    return (xlocs,ylocs)

def get_count(allcoords,l):
    longs = [x[0] for x in allcoords]
    lats  = [x[1] for x in allcoords]
    
    boundlong = (min(longs), max(longs))
    boundlat = (min(lats), max(lats))
    
    grid, _, _ = np.histogram2d(longs, lats,bins=generate_bin_edges((boundlong,boundlat),l))
    return np.count_nonzero(grid)

def get_dimension(scale,counts):
    slope, intercept, r_value, p_value, std_err = stats.linregress(np.log(scale),np.log(counts))

    dimension = -slope
    return dimension

def set_counts(new, ls):
    counts = [0]*len(ls)

    if new:
        print('Occupied Boxes')
        print('Intervals   ','Boxes')
        s1= "          "
        s2= "         "
        for i in range(len(ls)):
            counts[i] = get_count(new,ls[i])
            if ((i+1)>=10):
                print(i+1,s2,counts[i])
            if ((i+1)<10):
                print(i+1,s1,counts[i])
    return counts

def plot(ls, counts):
    #ignore UserWarning
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        plt.loglog(ls,counts,'bx')
        plt.grid("True")
        plt.xlabel('$\it{Log(ls)}$')
        plt.ylabel('$\it{Log(box-counts)}$')
        plt.show()
        print('Fractal Dimension:',get_dimension(ls,counts))

def draw_map(center, zoom):
    return Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom, layout= widgets.Layout(width='75%'))

def load_grid(df, m):
    grid = GeoData(geo_dataframe = df, style={})
    m.add(grid)
    return m

In [50]:
# MAIN
center = [38,-121]
zoom = 7
fp = 'sets/cali1.geojson'

title = widgets.HTML(
    value="<b>Fractal Dimension of Electrical Grid Calculator</b><br>Please create a polygon filter",
)
out1 = widgets.Output(layout=widgets.Layout(width='20%'))
out2 = widgets.Output()

#load cali grid in map
df = gpd.read_file(fp).drop(columns=['Creator_Date','Last_Editor_Date','Owner', 'kV_Sort', 'Type','Legend', 
                                     'Creator','Last_Editor', 'Comments', 'Source', 'TLine_Name', 'GlobalID'])
m = load_grid(df, draw_map(center, zoom))

#add draw polygon func
draw_control = DrawControl(edit=False, circlemarker={}, polyline={}, 
                        polygon = {
                            "shapeOptions": {
                                "fillColor": "#FF0000",
                                "color": "#FF0000",
                                "fillOpacity": 0.4
                            }}
                            )

def handle_draw(target, action, geo_json):
    dc = geo_json['geometry']

    if action in ['created']: 
        out1.clear_output()
        out2.clear_output()
        with out1:
                set = fill_set(filter_coords(dc, df))
                ls = np.logspace(-2,-5,10)
                counts = set_counts(set, ls)
        with out2:
            plot(ls, counts) 

draw_control.on_draw(handle_draw)

m.add(draw_control)

display(title)
display(widgets.HBox([m,out1]))
display(out2)

HTML(value='<b>Fractal Dimension of Electrical Grid Calculator</b><br>Please create a polygon filter')

Output()